In [1]:
# scraping the Ontario SIU Directors Reports from 2017 to 2018
# the reports to scrape are in an csv file in reports_to_scrape folder downloaded from their site

# import libraries
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import bs4
import re
from tqdm import tqdm

In [2]:
# import pages to scrape from the csv file
df = pd.read_csv("reports_to_scrape/on_siu_2018_2023_w_keywords.csv")
df.head()

,Unnamed: 0,Case Number,Date Report Signed,Link Text,URLs
0,2,23-PVI-099,11-Jul-23,Read Full Text,https://www.siu.on.ca/en/directors_report_deta...
1,6,23-OCI-091,10-Jul-23,Read Full Text,https://www.siu.on.ca/en/directors_report_deta...
2,10,23-OCI-083,10-Jul-23,Read Full Text,https://www.siu.on.ca/en/directors_report_deta...
3,13,23-OVI-078,07-Jul-23,Read Full Text,https://www.siu.on.ca/en/directors_report_deta...
4,14,23-OCI-077,07-Jul-23,Read Full Text,https://www.siu.on.ca/en/directors_report_deta...


In [3]:
# get the link from the case number part of the df

case_urls = df["URLs"].tolist()

In [4]:
case_urls[1]

'https://www.siu.on.ca/en/directors_report_details.php?drid=2488'

In [5]:
# scrape the first directors report 23-PVI-099 	

hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(case_urls[43],headers=hdr)
page = urlopen(req)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
print(soup.find("title"))

<title>Special Investigations Unit -- Director's Report Details, Case Number: 22-TCI-189</title>


In [6]:
def get_all_headings(soup):
    section_headers = soup.findAll(re.compile('^h[2-5]$'))
    
    def match_relevant_ids(section_header):
        header_id = section_header.get("id")
        
        if header_id == None:
            return False
        
        regex = re.compile('^section_[1-9]$')
        match = regex.search(section_header.get("id"))
        if match:
            return True
        else:
            return False
        
    section_headers = tuple(filter(match_relevant_ids, section_headers))
    
    return section_headers

In [7]:
def find_investigation_section(soup):
    try:
        investigation_title = soup.find("h2", string="The Investigation")


        next_sibling = investigation_title.next_sibling
        notification = [next_sibling]

        while next_sibling.next_sibling.name == "div":
            next_sibling = next_sibling.next_sibling
            notification.append(next_sibling)

        #print(notification)
        return notification
    except:
        return "Error finding Investigation Section"

In [8]:
# Information we are looking for:
# Province, city, police service, officer name, year, consequences for civilians (injuries), 
# consequences, if any, for officers (fine, dismissal, none, etc.), 
# investigation outcome (charge, acquittal complaint dismissal, etc.), and reason for police call. 

province = "Ontario"

# a lot of the information we might be looking for can be found in section 3 of the report
text = soup.find_all("h3")[5].next_sibling.get_text()
text

'On July 28, 2022, at 1:08 p.m., the Toronto Police Service ('

In [9]:
def find_year(soup):
    text = str(find_investigation_section(soup))
    if text != "Error finding Investigation Section":
        match = re.search(r"(20\d{2})", text)
        if match is not None:
            return match.group(1)
    else:
        return text

year = find_year(soup)
print(year)

2022


In [10]:
# list from https://www.oacp.ca/en/about-us/ontario-police-organizations.aspx
ontario_police_services = [
    'Akwesasne Mohawk Police Service',
    'Anishinabek Police Service',
    'Aylmer Police Service',
    'Barrie Police Service',
    'Belleville Police Service',
    'Brantford Police Service',
    'Brockville Police Service',
    'Chatham Kent Police Service',
    'City of Kawartha Lakes Police Service',
    'Cobourg Police Service',
    'Cornwall Police Service',
    'Deep River Police Service',
    'Durham Regional Police Service',
    'Dryden Police Service',
    'Greater Sudbury Police Service',
    'Guelph Police Service',
    'Halton Regional Police Service',
    'Hamilton Police Service',
    'Hanover Police Service',
    'Kingston Police',
    'Lac Seul Police Service',
    'LaSalle Police Service',
    'London Police Service',
    'Niagara Parks Police',
    'Niagara Regional Police Service',
    'Nishnawbe-Aski Police Service',
    'North Bay Police Service',
    'OPP',
    'Ontario Provincial Police',
    'Owen Sound Police Service',
    'Ottawa Police Service',
    'Peterborough Police Service',
    'Peel Regional Police',
    'Port Hope Police Service',
    'Rama Police Service',
    'Sarnia Police Service',
    'RCMP',
    'Royal Canadian Mounted Police',
    'Sault Ste. Marie Police Service',
    'Saugeen Shores Police Service',
    'Smiths Falls Police Service',
    'Six Nations Police Service',
    'St. Thomas Police Service',
    'Strathroy-Caradoc Police Service',
    'South Simcoe Police Service',
    'Timmins Police Service',
    'Stratford Police Service',
    'Thunder Bay Police Service',
    'Treaty Three Police Service',
    'Toronto Police Service',
    'Waterloo Regional Police Service',
    'U.C.C.M. Anishnaabe Police',
    'Wikwemikong Tribal Police Service',
    'West Grey Police Service',
    'Woodstock Police Service',
    'Windsor Police Service',
    'York Regional Police'
]

In [11]:
def find_police_service(soup):
    text = str(find_investigation_section(soup))
    if text != "Error finding Investigation Section":
        for service_name in ontario_police_services:
            if service_name in text:
                return service_name
    else:
        return text

In [12]:
police = find_police_service(soup)
print(police)
#print(text)

Toronto Police Service


In [13]:
municipalities_df = pd.read_csv("mmah-list-of-ontario-municipalities-en-utf8-2022-10-05.csv")

def get_municipality_name(tag):
    a_soup = BeautifulSoup(tag, "html.parser")
    
    if isinstance(a_soup, bs4.element.Tag):
        full_name = a_soup.get_text()
    else:
        full_name = tag
            
    municipality_name = ", ".join(full_name.split(", ")[:-1])
    return municipality_name
    

list_of_municipalities_in_ontario = list(municipalities_df["Municipality"].apply(get_municipality_name))

list_of_municipalities_in_ontario = list(filter(lambda x: True if len(x) > 0 else False, list_of_municipalities_in_ontario))

In [14]:
def find_incident_narrative():
    try:
        
        if soup.find("h2", string="Incident Narrative"):
            incident_narrative_title = soup.find("h2", string="Incident Narrative")
        elif soup.find("h2", string="Incident narrative"): 
            incident_narrative_title = soup.find("h2", string="Incident Narrative")
        elif soup.find("h2", string="Event Chronology"):
            incident_narrative_title = soup.find("h2", string="Event Chronology")
        else:
            incident_narrative_title = soup.find("h2", string="The Investigation")
        
        
        next_sibling = incident_narrative_title.next_sibling
        incident_narrative = [next_sibling]

        while next_sibling.next_sibling.name == "div":
            next_sibling = next_sibling.next_sibling
            incident_narrative.append(next_sibling)

        #print(incident_narrative)
        return incident_narrative
    except:
        print("Issue with finding Incident Narrative")
        return "Issue with finding Incident Narrative"

In [15]:
def find_city(soup):
    incident_narratives = find_incident_narrative()
    if incident_narratives != "Issue with finding Incident Narrative":    
        for p in incident_narratives:
            for municipality in list_of_municipalities_in_ontario:
                regex = r"\b(?=\w)" + re.escape(municipality) + r"\b(?!\w)"
                match = re.search(regex, p.get_text(), re.IGNORECASE)
                if match:
                    return municipality
    else:
        return "Municipality not found"

In [16]:
# officer name is not included in accordance with section 21 of FIPPA
# moving onto civilian consquences, this can be found in the text we have
# apparent its not always consistent where this info is
    
from string import punctuation

def find_civilian_consequences(soup):
    
    keywords = ['death', 'injury', 'fracture', 'diagnosed', 
                'diagnoses', 'dead', 'post-mortem', 'cause of death', 
                'injuries', 'CT scan', 'assessed', 'assessment', 'treated']
    
    try:
    
        if soup.find("h3", string = "Nature of Injuries / Treatment"):
            temp = soup.find("h3", string = "Nature of Injuries / Treatment")
        elif soup.find("h3", string = "Nature of Injury/Treatment"): 
            temp = soup.find("h3", string = "Nature of Injury/Treatment")
        elif soup.find("h3", string = "Cause of Death"):
            temp = soup.find("h3", string = "Cause of Death")
        elif soup.find("h3", string = "Cause of death"):
            temp = soup.find("h3", string = "Cause of death")
        elif soup.find("h3", string = "Nature of Injuries / Treatment/Cause of Death"):
            temp = soup.find("h3", string = "Nature of Injuries / Treatment/Cause of Death")
        elif soup.find("h3", string = "Nature of injury/treatment"):
            temp = soup.find("h3", string = "Nature of injury/treatment")
        else:
            temp = soup.find("h2", string="Incident Narrative")

        next_sibling = temp.next_sibling
        incident = [next_sibling]


        while next_sibling.next_sibling.name == "div":
            next_sibling = next_sibling.next_sibling
            incident.append(next_sibling.text)

        civilian_consequences = ""

        CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        cleantext = re.sub(CLEANR, '', str(incident))
        temp = cleantext.split('.')
        for i in temp:
            for j in keywords:
                if j in i:
                    civilian_consequences = i

        if civilian_consequences == "":
            civilian_consequences = cleantext

        civilian_consequences=[i.strip(punctuation) for i in civilian_consequences.split()]
        civilian_consequences = " ".join(civilian_consequences)
    
    except:
        print(soup.find("title"))
        civilian_consequences = "Error when extracting consequences."
        # I think it is because they use the Event Chronology title instead 
        # but I want to see how many use it before fixing it all over
    
    return civilian_consequences

In [17]:
print(find_civilian_consequences(soup))

The Complainant was transported to hospital following his arrest and diagnosed with a right rib fracture


In [18]:
# find consequences for officer
# personally, I can't find an area of these reports that mention consequences for the officers

In [19]:
def find_decision_date(soup):
    body = soup.find("div", class_="body-field")
    decision_date_element = body.contents[-3]
    decision_date = decision_date_element.string
    
    if decision_date is not None:
        decision_date = decision_date.replace("\xa0", " ")

In [20]:
# investigation outcome
def find_investigation_outcome(soup):
    # investigation outcome
    # this is found in the director's analysis section/last paragraph of the report
    all_h2 = soup.find_all("h2")
    
    mandate = None
    for h2 in all_h2:
        if h2.find(string=re.compile("Mandate of the")):
            mandate = h2
            break
            
    if mandate is None:
        return None
    
    keywords = ['basis', 'reasonable grounds']
    decision = ""
    
    for i in mandate.parent.contents:
    
#     decision_paragraph = mandate.parent.contents[-9]
        decision_final_paragraph_text = i.get_text()
    
        #print(decision_final_paragraph_text)

        CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        cleantext = re.sub(CLEANR, '', str(decision_final_paragraph_text))
        temp = cleantext.split('.')
        for i in temp:
            for j in keywords:
                if j in i:
                    decision = i

    if decision == "":
        decision = cleantext

    decision=[i.strip(punctuation) for i in decision.split()]
    decision = " ".join(decision)

    return decision

In [21]:
find_investigation_outcome(soup)

'As such there is no basis for proceeding with criminal charges'

In [22]:
# reason for police investigation
# not really sure about this one


In [ ]:
results = []
for case in tqdm(case_urls):
    
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(case,headers=hdr)
    page = urlopen(req)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    
    province = "Ontario"
    
    year = find_year(soup)
    police_service = find_police_service(soup)
    
    civilian_consequences = find_civilian_consequences(soup)
    
    city = find_city(soup)
    
    investigation_outcome = find_investigation_outcome(soup)
    
    results.append([year, police_service, city, province, civilian_consequences, investigation_outcome, case])
    
results_df = pd.DataFrame(results, columns=['Year',  'Police Service', 'City', 'Province', 'Civilian Consequences', 'Investigation Final Decision', 'Link'])
    

 37%|██████████████████████████▏                                            | 257/697 [01:02<02:01,  3.63it/s]

<title>Special Investigations Unit -- Director's Report Details, Case Number: 20-TCD-124</title>


 67%|███████████████████████████████████████████████▎                       | 464/697 [01:51<00:59,  3.92it/s]

<title>Special Investigations Unit -- Director's Report Details, Case Number: 19-TCI-073a</title>


 74%|████████████████████████████████████████████████████▋                  | 517/697 [02:03<00:34,  5.25it/s]

<title>Special Investigations Unit -- Director's Report Details, Case Number: 18-OCI-342</title>
Issue with finding Incident Narrative


 92%|████████████████████████████████████████████████████████████████▉      | 638/697 [02:31<00:14,  4.20it/s]

In [ ]:
results_df

In [ ]:
results_df.Year = pd.to_numeric(results_df.Year, errors='coerce').astype('Int64')
results_df = results_df[((results_df.Year > 2009) & (results_df.Year < 2021))]
results_df

In [ ]:
results_df.to_csv("results_on_2018_2023.csv", index=False)